<a href="https://colab.research.google.com/github/Romasha-Guin/Edulyt-India/blob/main/Problem_Statement_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem Statement 2:**

In [347]:
import numpy as np
import pandas as pd

Load the dataset.

In [348]:
df_1 = pd.read_csv('Customer_Info.csv')
df_2 = pd.read_csv('Project_2.csv')

In [349]:
df_1.head()

,C_ID,Email,Name,Mobile_number,Gender,Age,City,State,Address
0,3768.0,eddie@yahoomail.com,EDDIE,9.045258e+09,M,83.0,Louisville,Kentucky,Masked
1,4852.0,rose@hotmail.com,ROSE,8.834789e+09,F,87.0,Kansas City,Missouri,Masked
2,1174.0,amy@yahoomail.com,AMY,9.557690e+09,F,31.0,Seattle,Washington,Masked
3,4807.0,clarence@gmail.com,CLARENCE,9.394398e+09,M,37.0,Seattle,Washington,Masked
4,9131.0,johnny@hotmail.com,JOHNNY,9.976624e+09,F,80.0,Columbus,Ohio,Masked


In [350]:
df_2.head()

,Credit_card,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,GTIN,MPN,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date
0,4595.0,534,SHOES,New,RKFCM,"$1,484.74","$1,471.74",BN710,26/08/2014,17:16:17,3109172,547836726,Peninsula Beauty,YE5901,Prepaid card,30462,0,NaN
1,6679.0,396,LUGGAGE,New,TNVON,"$1,452.50","$1,442.50",KF815,26/01/2014,15:40:52,7583605,279617132,DODOcase,UW7354,Mobile carrier Billing,37927,0,NaN
2,7075.0,449,SHOES,Refurbished,VKKEA,"$4,308.02","$4,284.02",UG960,03/01/2014,01:29:05,9985192,633765867,Staples,VN6656,Credit card,80640,0,NaN
3,1418.0,145,BEDDING,New,PIUEM,"$3,272.59","$3,249.59",DK471,30/05/2014,09:10:00,7720605,734500382,Road Runner Sports,MS6579,Prepaid card,58692,0,NaN
4,6740.0,844,ELECTRONICS,Used,FMBZC,$628.67,$608.67,OJ311,14/08/2014,05:31:30,8367528,241944637,L'Occitane,GN3944,Debit card,21505,0,NaN


**Sanity Checks – Data Cleaning**

In [351]:
#To replace missing values.
df_1.fillna(99999, inplace=True)

In [352]:
df_2.columns

Index(['Credit_card', 'Product_ID', 'P_CATEGORY', 'CONDTION', 'Brand',
       'Price ', 'Selling_price', 'Coupon_ID', 'Date', 'Time', 'GTIN', 'MPN',
       'Merchant_name', 'M_ID', 'Payment Method', 'Transaction ID',
       'Return_ind', 'Return_date'],
      dtype='object')

In [353]:
#To apply an automatic discount of 5% to entries where the "Price" is equal to the "Selling Price."
df_2['Price '] = df_2['Price '].replace('[\$,]', '', regex=True).astype(str).str.replace('(\(|\))', '', regex=True).astype(float)
df_2['Selling_price'] = df_2['Selling_price'].replace('[\$,]', '', regex=True).astype(str).str.replace('(\(|\))', '', regex=True).astype(float)

mask = df_2['Price '] == df_2['Selling_price']
df_2.loc[mask, 'Price '] *= 0.95

In [354]:
#To ensure that the return date is after the purchase date.
df_2['Return_date'] = pd.to_datetime(df_2['Return_date'])
df_2['Date'] = pd.to_datetime(df_2['Date'])

df_2 = df_2[df_2['Return_date'] >= df_2['Date']]

<ipython-input-354-a9bdfee01cc8>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_2['Return_date'] = pd.to_datetime(df_2['Return_date'])
<ipython-input-354-a9bdfee01cc8>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_2['Date'] = pd.to_datetime(df_2['Date'])


In [355]:
#To ensure that no discount is given and the selling price is equal to the price when the Coupon ID is NULL.
mask = df_2['Coupon_ID'].isnull()
df_2.loc[mask, 'Selling_price'] = df_2.loc[mask, 'Price ']

In [356]:
#To ensure that age greater is than 18 for CC holders.
df_1 = df_1[df_1['Age'] > 18]

In [357]:
#To ensure that the transaction ID is unique.
is_unique = df_2['Transaction ID'].duplicated().any()
if is_unique:
    print("Transaction ID is not unique.")
else:
    print("Transaction ID is unique.")


Transaction ID is not unique.


**Tasks:**

In [358]:
#To perform customer segmentation based on age.
young_female_range = (18, 35)
mid_age_female_range = (36, 50)
old_female_range = (50, float('inf'))
young_male_range = (18, 35)
mid_age_male_range = (36, 50)
old_male_range = (50, float('inf'))

df_1['Segment'] = ''

df_1.loc[(df_1['Age'] >= young_female_range[0]) & (df_1['Age'] <= young_female_range[1]) & (df_1['Gender'] == 'F'), 'Segment'] = 'Young Females'
df_1.loc[(df_1['Age'] >= mid_age_female_range[0]) & (df_1['Age'] <= mid_age_female_range[1]) & (df_1['Gender'] == 'F'), 'Segment'] = 'Mid-Age Females'
df_1.loc[(df_1['Age'] >= old_female_range[0]) & (df_1['Age'] <= old_female_range[1]) & (df_1['Gender'] == 'F'), 'Segment'] = 'Old Females'
df_1.loc[(df_1['Age'] >= young_male_range[0]) & (df_1['Age'] <= young_male_range[1]) & (df_1['Gender'] == 'M'), 'Segment'] = 'Young Males'
df_1.loc[(df_1['Age'] >= mid_age_male_range[0]) & (df_1['Age'] <= mid_age_male_range[1]) & (df_1['Gender'] == 'M'), 'Segment'] = 'Mid-Age Males'
df_1.loc[(df_1['Age'] >= old_male_range[0]) & (df_1['Age'] <= old_male_range[1]) & (df_1['Gender'] == 'M'), 'Segment'] = 'Old Males'


<ipython-input-358-0666724dcb98>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Segment'] = ''


In [359]:
df_1.head()

,C_ID,Email,Name,Mobile_number,Gender,Age,City,State,Address,Segment
0,3768.0,eddie@yahoomail.com,EDDIE,9.045258e+09,M,83.0,Louisville,Kentucky,Masked,Old Males
1,4852.0,rose@hotmail.com,ROSE,8.834789e+09,F,87.0,Kansas City,Missouri,Masked,Old Females
2,1174.0,amy@yahoomail.com,AMY,9.557690e+09,F,31.0,Seattle,Washington,Masked,Young Females
3,4807.0,clarence@gmail.com,CLARENCE,9.394398e+09,M,37.0,Seattle,Washington,Masked,Mid-Age Males
4,9131.0,johnny@hotmail.com,JOHNNY,9.976624e+09,F,80.0,Columbus,Ohio,Masked,Old Females


In [360]:
#To calculate the spend in terms of product, state, and payment method.
spend_summary = df_2.groupby(['P_CATEGORY', 'CONDTION', 'Payment Method'])['Price '].sum().reset_index()
spend_summary = spend_summary.sort_values('Price ', ascending=False)

In [361]:
spend_summary

,P_CATEGORY,CONDTION,Payment Method,Price
54,COMPUTERS,New,Mobile carrier Billing,101829.2475
71,DECOR,Refurbished,Mobile carrier Billing,79830.0700
127,OFFICE SUPPLIES,Refurbished,Mobile carrier Billing,73039.6500
122,OFFICE SUPPLIES,New,Credit card,72007.9200
52,COMPUTERS,New,Credit card,70220.3300
...,...,...,...,...
91,GAMES,New,Debit card,1680.3300
57,COMPUTERS,Refurbished,Debit card,1435.7700
99,GAMES,Used,Debit card,1017.7700
133,SHOES,New,Debit card,645.1000


In [362]:
#To calculate the highest 5 spending.
highest_spend = spend_summary.nlargest(5, 'Price ')

In [363]:
highest_spend

,P_CATEGORY,CONDTION,Payment Method,Price
54,COMPUTERS,New,Mobile carrier Billing,101829.2475
71,DECOR,Refurbished,Mobile carrier Billing,79830.0700
127,OFFICE SUPPLIES,Refurbished,Mobile carrier Billing,73039.6500
122,OFFICE SUPPLIES,New,Credit card,72007.9200
52,COMPUTERS,New,Credit card,70220.3300


In [364]:
column_rename_mapping = {
    'Price ': 'Price'
}
df_2.rename(columns=column_rename_mapping, inplace=True)

In [365]:
#Calculating the discount for each transaction.
df_2['Price'] = df_2['Price'].astype(str)
df_2['Selling_price'] = df_2['Selling_price'].astype(str)

df_2['Price'] = df_2['Price'].replace('[\$,]', '', regex=True).str.replace('(\(|\))', '', regex=True).astype(float)
df_2['Selling_price'] = df_2['Selling_price'].replace('[\$,]', '', regex=True).str.replace('(\(|\))', '', regex=True).astype(float)

df_2['Discount'] = (df_2['Price'] - df_2['Selling_price']) / df_2['Price'] * 100

In [366]:
#To analyze the return category based on state, age group, condition, product category, and discount.
df_1['C_ID'] = df_1['C_ID'].astype(str)
df_2['Credit_card'] = df_2['Credit_card'].astype(str)

merged_df = pd.merge(df_1, df_2, left_on='C_ID', right_on='Credit_card', how='inner')

return_category_summary = merged_df.groupby(['Segment', 'State', 'CONDTION', 'P_CATEGORY', 'Discount'])['Return_ind'].sum().reset_index()
return_category_summary = return_category_summary.sort_values('Return_ind', ascending=False)

In [367]:
return_category_summary

,Segment,State,CONDTION,P_CATEGORY,Discount,Return_ind
123,Old Females,California,New,KITCHEN & DINING,2.540232,3
383,Old Females,Washington,Refurbished,GAMES,0.825767,2
324,Old Females,Texas,New,OFFICE SUPPLIES,0.926813,2
338,Old Females,Texas,Refurbished,DECOR,0.501834,2
339,Old Females,Texas,Refurbished,DECOR,0.521907,2
...,...,...,...,...,...,...
76,Old Females,Arizona,Refurbished,BABY CLOTHING,1.419433,1
75,Old Females,Arizona,Refurbished,BABY CLOTHING,1.176629,1
74,Old Females,Arizona,New,SHOES,1.703570,1
73,Old Females,Arizona,New,SHOES,0.779959,1


In [368]:
#To create a profile of customers in terms of the timing of their orders.
df_2['Date'] = pd.to_datetime(df_2['Date'])
df_2['Time'] = pd.to_datetime(df_2['Time'])

df_2['Order_day_of_week'] = df_2['Date'].dt.day_name()
df_2['Order_hour'] = df_2['Time'].dt.hour

customer_order_profile = df_2.groupby('M_ID').agg(Average_order_hour=('Order_hour', 'mean'),
                                                 Most_common_order_day=('Order_day_of_week', lambda x: x.value_counts().index[0]))

In [369]:
customer_order_profile

,Average_order_hour,Most_common_order_day
M_ID,,
EI6773,13.480000,Monday
EI6774,8.680000,Wednesday
EI6775,14.363636,Tuesday
EX7729,11.073171,Friday
EX7730,10.478261,Thursday
EX7731,15.250000,Monday
EY1658,10.600000,Tuesday
EY1659,12.483871,Saturday
EY1660,11.583333,Tuesday


In [370]:
#To find out which payment method is providing more discount for customers.
average_discount_by_payment_method = df_2.groupby('Payment Method')['Discount'].mean().reset_index()
average_discount_by_payment_method = average_discount_by_payment_method.sort_values('Discount', ascending=False)

In [371]:
average_discount_by_payment_method

,Payment Method,Discount
4,Mobile carrier Billing,3.251006
1,Debit card,2.658885
0,Credit card,2.401257
2,Deirect debits,1.772666
6,Prepaid card,1.765150
3,Gift card,1.578962
5,Paypal wallet,1.530456


In [372]:
#To create a profile for high-value items vs. low-value items.
high_value_threshold = 1000
df_2['Item_value'] = df_2['Selling_price'].apply(lambda price: 'High-value' if price > high_value_threshold else 'Low-value')
item_value_orders = df_2.groupby('Item_value')['Transaction ID'].count().reset_index()

In [373]:
item_value_orders

,Item_value,Transaction ID
0,High-value,894
1,Low-value,216
